# Tutorial data

In [ ]:
# import pandas as pd

# data_path = "data/train/day_0_0.001_train.csv"

# df = pd.read_csv(data_path, header=None)
# df.head()

In [ ]:
# example = df.loc[1, 0]
# example

In [ ]:
# features = example.split('\t')
# len(features)
# features

# Read s3 orc data

In [1]:
column_names = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        column_names.append(line.split(' ')[1].strip())
print(column_names)

['_11001', '_11002', '_11003', '_11004', '_11007', '_11008', '_11021', '_11022', '_11023', '_11024', '_11041', '_11042', '_11043', '_11044', '_11045', '_11046', '_11061', '_11062', '_11063', '_11064', '_11065', '_11066', '_11081', '_11082', '_11083', '_11084', '_11085', '_11086', '_11601', '_11602', '_11603', '_12001', '_12002', '_12003', '_12004', '_12005', '_12006', '_20001', '_20002', '_20003', '_20101', '_20102', '_20201', '_20202', '_20203', '_20204', '_20205', '_20206', '_20207', '_20208', '_20209', '_20210', '_30001', '_30002', '_30003', '_30004', '_30005', '_30006', '_30201', '_30202', '_30203', '_30204', '_30205', '_30206', '_30207', '_40001', '_40002', '_40003', '_40004', '_40005', '_40201', '_40202', '_40203', '_40204', '_40205', '_40206', '_40207', '_40208', '_40209', '_40210', '_40211', '_40212', '_40213', '_40214', '_40215', '_40231', '_40301', '_40302', '_40303', '_40304', '_40305', '_40306', '_40307', '_40321', '_40322', '_40323', '_40324', '_50801', '_50802', '_50805',

In [2]:
import metaspore as ms

# train_dataset_path = ROOT_DIR + '/data/train/day_0_0.001_train.csv'

file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 20
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

# train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/part-00000-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc'
# train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'

spark_confs = {
    'spark.eventLog.enabled':'true',
    'spark.executor.memory': '10g',
    'spark.driver.memory': '10g',
    "spark.driver.maxResultSize": "10g",
    "spark.sql.files.ignoreCorruptFiles": "true"
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=2,
                                     server_count=2,
                                     log_level='WARN',
                                     spark_confs=spark_confs)

train_dataset = ms.input.read_s3_csv(spark_session, 
                                     train_dataset_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=column_names,
                                     multivalue_column_names=column_names[:-1])

# train_dataset = spark_session.read.orc(train_dataset_path)

# train_dataset.printSchema()

/usr/local/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-21 11:37:27,289 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


ignore shuffle


In [3]:
import pandas as pd
pd.set_option("display.max_columns", None)

df = train_dataset.toPandas()
df.head()

2024-07-21 11:37:34,086 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,_11001,_11002,_11003,_11004,_11007,_11008,_11021,_11022,_11023,_11024,_11041,_11042,_11043,_11044,_11045,_11046,_11061,_11062,_11063,_11064,_11065,_11066,_11081,_11082,_11083,_11084,_11085,_11086,_11601,_11602,_11603,_12001,_12002,_12003,_12004,_12005,_12006,_20001,_20002,_20003,_20101,_20102,_20201,_20202,_20203,_20204,_20205,_20206,_20207,_20208,_20209,_20210,_30001,_30002,_30003,_30004,_30005,_30006,_30201,_30202,_30203,_30204,_30205,_30206,_30207,_40001,_40002,_40003,_40004,_40005,_40201,_40202,_40203,_40204,_40205,_40206,_40207,_40208,_40209,_40210,_40211,_40212,_40213,_40214,_40215,_40231,_40301,_40302,_40303,_40304,_40305,_40306,_40307,_40321,_40322,_40323,_40324,_50801,_50802,_50805,_50806,_50807,_50810,_51001,_51002,_51003,_51004,_51005,_51006,_51011,_51012,_51013,_51014,_51021,_51022,_51023,_51024,_51025,_51026,_51031,_51032,_51033,_51034,_51035,_51036,_51041,_51042,_51043,_51044,_51045,_51046,_52001,_52002,_52003,_52004,_52005,_61101,_61102,_61103,_70001,_70002,_70003,_70004,_70005,_70006,_70007,_70008,_70009,_70010,_70011,_70031,_70032,_70033,_70034,_70035,_70036,_70037,_70038,_70039,_70040,_70041,_70301,_70302,_70303,_70304,_70305,_70306,_70601,_70602,_70603,_70604,_70605,_70606,_70901,_70902,_70903,_70904,_70905,_70906,_72401,_72402,_80001,_80002,_80003,_80004,_80005,_80006,_80007,_80008,_80009,_80010,_80011,_80012,_80013,_80014,_80015,_80016,_80017,_80018,_80019,_80020,_80021,_80022,_80023,_80024,_80025,_80026,_80027,_80028,_80029,_80030,_80031,_80032,_80033,_80034,_80035,_80036,_80037,_80038,_80039,_80040,_80041,_80042,_80043,_80044,_80045,_80046,_80047,_80048,_80049,_80050,_80051,_80052,_80053,_80054,_80055,_80056,_80057,_80058,_80059,_80060,_80061,_80062,_80063,_80064,_80065,_80066,_80067,_80068,_80069,_80070,_80071,_80072,_80073,_80074,_80075,_80076,_80077,_80078,_80079,_80080,_80081,_80082,_80083,_80084,_80085,_80086,_80087,_80088,_80089,_80090,_80091,_80092,_80093,_80094,_80095,_80096,_80097,_80098,_80099,_80100,_80101,_80102,_80103,_80104,_80105,_80106,_80107,_80108,_80109,_80110,_80111,_80112,_80113,_80114,_80115,_80125,_80126,_80127,_80128,_80129,_80130,_80131,_80132,_80133,_80134,_80135,_80136,_80137,_80138,_80139,_80140,_80141,_80142,_80143,_80144,_80145,_80146,_80147,_80148,_80149,_80150,_80151,label
0,"[38769583577632851470.6931471824645996, 39603...","[145793842084348162990.6931471824645996, 3615...",[1187569319056951.3862943649291992],[1187569319062944.060442924499512],"[145793842084348162990.6931471824645996, 3615...",[1187569319390604.110873699188232],[],[],[],[],[39616979043979942990.6931471824645996],"[144052759590420437910.6931471824645996, 3961...","[39583930304135115260.6931471824645996, 53416...",[1187569322670720.6931471824645996],[1187569653783872.4849066734313965],[1187569653711261.0986123085021973],[],"[144072535572195799010.6931471824645996, 3702...","[37028272254858111300.6931471824645996, 39616...",[],[1187569318241191.0986123085021973],[1187569654697721.0986123085021973],"[144072535572195799011.0986123085021973, 3702...","[144072535572195799011.945910096168518, 14580...","[141431098596062431110.6931471824645996, 1440...",[1187569654371062.1972246170043945],[1187569654438934.077537536621094],[1187569654466404.127134323120117],"[3113719539837061.7917594909667969, 311371953...",[],[3113719539837060.6931471824645996],"[144052759590420437910.6931471824645996, 1440...","[141431098596062431110.6931471824645996, 3958...","[143152167126395678210.6931471824645996, 3788...",[1187785070326513.0910425186157227],[1187785070080721.0986123085021973],[1187785070160092.3025851249694824],[609263171274291.0],[609264007680991.0],[609263171171131.0],[609258213882141.0],[609261178775011.0],[609146718144861.0],[41983727253686639581.0],[634174621597341.0],[609261024559911.0],[609261033396151.0],[609261024475101.0],[609261031903621.0],[609261024068361.0],[609261033745311.0],[609261132725481.0],[620638256379631.0],[620638255203991.0],[645207868698881.0],[31153

In [4]:
num_positive = df[df['label']=='1'].shape[0]
num_negative = df[df['label']=='0'].shape[0]
print(f"num_positive: {num_positive}, num_negative: {num_negative}, ratio: {num_positive/num_negative*100}")

num_positive: 6060, num_negative: 247428, ratio: 2.4491973422571416


In [5]:
from tqdm import tqdm

columns = df.columns.tolist()
print(columns)

total_hash = 0
feature_hash = {}
feature_value = {}

for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    for column in columns[:-1]:
        if column in feature_value:
            feature_value[column].append(row[column].shape[0])
        else:
            feature_value[column] = [row[column].shape[0]]
        
        for item in row[column]:
            if '\003' in item:
                hash, weight = item.split('\003')
            else:
                hash, weight = '', 0
            if column in feature_hash:
                feature_hash[column].add(hash)
            else:
                feature_hash[column] = set(hash)

df_feature = pd.DataFrame(feature_value)
df_feature.describe()

['_11001', '_11002', '_11003', '_11004', '_11007', '_11008', '_11021', '_11022', '_11023', '_11024', '_11041', '_11042', '_11043', '_11044', '_11045', '_11046', '_11061', '_11062', '_11063', '_11064', '_11065', '_11066', '_11081', '_11082', '_11083', '_11084', '_11085', '_11086', '_11601', '_11602', '_11603', '_12001', '_12002', '_12003', '_12004', '_12005', '_12006', '_20001', '_20002', '_20003', '_20101', '_20102', '_20201', '_20202', '_20203', '_20204', '_20205', '_20206', '_20207', '_20208', '_20209', '_20210', '_30001', '_30002', '_30003', '_30004', '_30005', '_30006', '_30201', '_30202', '_30203', '_30204', '_30205', '_30206', '_30207', '_40001', '_40002', '_40003', '_40004', '_40005', '_40201', '_40202', '_40203', '_40204', '_40205', '_40206', '_40207', '_40208', '_40209', '_40210', '_40211', '_40212', '_40213', '_40214', '_40215', '_40231', '_40301', '_40302', '_40303', '_40304', '_40305', '_40306', '_40307', '_40321', '_40322', '_40323', '_40324', '_50801', '_50802', '_50805',

100%|██████████| 253488/253488 [07:16<00:00, 580.57it/s]


,_11001,_11002,_11003,_11004,_11007,_11008,_11021,_11022,_11023,_11024,_11041,_11042,_11043,_11044,_11045,_11046,_11061,_11062,_11063,_11064,_11065,_11066,_11081,_11082,_11083,_11084,_11085,_11086,_11601,_11602,_11603,_12001,_12002,_12003,_12004,_12005,_12006,_20001,_20002,_20003,_20101,_20102,_20201,_20202,_20203,_20204,_20205,_20206,_20207,_20208,_20209,_20210,_30001,_30002,_30003,_30004,_30005,_30006,_30201,_30202,_30203,_30204,_30205,_30206,_30207,_40001,_40002,_40003,_40004,_40005,_40201,_40202,_40203,_40204,_40205,_40206,_40207,_40208,_40209,_40210,_40211,_40212,_40213,_40214,_40215,_40231,_40301,_40302,_40303,_40304,_40305,_40306,_40307,_40321,_40322,_40323,_40324,_50801,_50802,_50805,_50806,_50807,_50810,_51001,_51002,_51003,_51004,_51005,_51006,_51011,_51012,_51013,_51014,_51021,_51022,_51023,_51024,_51025,_51026,_51031,_51032,_51033,_51034,_51035,_51036,_51041,_51042,_51043,_51044,_51045,_51046,_52001,_52002,_52003,_52004,_52005,_61101,_61102,_61103,_70001,_70002,_70003,_70004,_70005,_70006,_70007,_70008,_70009,_70010,_70011,_70031,_70032,_70033,_70034,_70035,_70036,_70037,_70038,_70039,_70040,_70041,_70301,_70302,_70303,_70304,_70305,_70306,_70601,_70602,_70603,_70604,_70605,_70606,_70901,_70902,_70903,_70904,_70905,_70906,_72401,_72402,_80001,_80002,_80003,_80004,_80005,_80006,_80007,_80008,_80009,_80010,_80011,_80012,_80013,_80014,_80015,_80016,_80017,_80018,_80019,_80020,_80021,_80022,_80023,_80024,_80025,_80026,_80027,_80028,_80029,_80030,_80031,_80032,_80033,_80034,_80035,_80036,_80037,_80038,_80039,_80040,_80041,_80042,_80043,_80044,_80045,_80046,_80047,_80048,_80049,_80050,_80051,_80052,_80053,_80054,_80055,_80056,_80057,_80058,_80059,_80060,_80061,_80062,_80063,_80064,_80065,_80066,_80067,_80068,_80069,_80070,_80071,_80072,_80073,_80074,_80075,_80076,_80077,_80078,_80079,_80080,_80081,_80082,_80083,_80084,_80085,_80086,_80087,_80088,_80089,_80090,_80091,_80092,_80093,_80094,_80095,_80096,_80097,_80098,_80099,_80100,_80101,_80102,_80103,_80104,_80105,_80106,_80107,_80108,_80109,_80110,_80111,_80112,_80113,_80114,_80115,_80125,_80126,_80127,_80128,_80129,_80130,_80131,_80132,_80133,_80134,_80135,_80136,_80137,_80138,_80139,_80140,_80141,_80142,_80143,_80144,_80145,_80146,_80147,_80148,_80149,_80150,_80151
count,253488.000000,253488.000000,253488.0,253488.0,253488.000000,253488.0,253488.000000,253488.000000,253488.0,253488.0,253488.000000,253488.000000,253488.000000,253488.0,253488.0,253488.0,253488.000000,253488.000000,253488.000000,253488.0,253488.0,253488.0,253488.000000,253488.000000,253488.000000,253488.0,253488.0,253488.0,253488.000000,253488.000000,253488.000000,253488.000000,253488.000000,253488.000000,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.000000,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.000000,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253488.0,253

In [6]:
feature_hash_count = dict([(k, [len(v)]) for k, v in feature_hash.items()])
df_hash = pd.DataFrame(feature_hash_count)
df_hash

,_11001,_11002,_11003,_11004,_11007,_11008,_11021,_11022,_11023,_11024,_11041,_11042,_11043,_11044,_11045,_11046,_11061,_11062,_11063,_11064,_11065,_11066,_11081,_11082,_11083,_11084,_11085,_11086,_11601,_11602,_11603,_12001,_12002,_12003,_12004,_12005,_12006,_20001,_20002,_20003,_20101,_20102,_20201,_20202,_20203,_20204,_20205,_20206,_20207,_20208,_20209,_20210,_30001,_30002,_30003,_30004,_30005,_30006,_30201,_30202,_30203,_30204,_30205,_30206,_30207,_40001,_40002,_40003,_40004,_40005,_40201,_40202,_40203,_40204,_40205,_40206,_40207,_40208,_40209,_40210,_40211,_40212,_40213,_40214,_40215,_40231,_40301,_40302,_40303,_40304,_40305,_40306,_40307,_40321,_40322,_40323,_40324,_50801,_50802,_50805,_50806,_50807,_50810,_51001,_51002,_51003,_51004,_51005,_51006,_51011,_51012,_51013,_51014,_51021,_51022,_51023,_51024,_51025,_51026,_51031,_51032,_51033,_51034,_51035,_51036,_51041,_51042,_51043,_51044,_51045,_51046,_52001,_52002,_52003,_52004,_52005,_61101,_61102,_61103,_70001,_70002,_70003,_70004,_70005,_70006,_70007,_70008,_70009,_70010,_70011,_70031,_70032,_70033,_70034,_70035,_70036,_70037,_70038,_70039,_70040,_70041,_70301,_70302,_70303,_70304,_70305,_70306,_70601,_70602,_70603,_70604,_70605,_70606,_70901,_70902,_70903,_70904,_70905,_70906,_72401,_72402,_80001,_80002,_80003,_80004,_80005,_80006,_80007,_80008,_80009,_80010,_80011,_80012,_80013,_80014,_80015,_80016,_80017,_80018,_80019,_80020,_80021,_80022,_80023,_80024,_80025,_80026,_80027,_80028,_80029,_80030,_80031,_80032,_80033,_80034,_80035,_80036,_80037,_80038,_80039,_80040,_80041,_80042,_80043,_80044,_80045,_80046,_80047,_80048,_80049,_80050,_80051,_80052,_80053,_80054,_80055,_80056,_80057,_80058,_80059,_80060,_80061,_80062,_80063,_80064,_80065,_80066,_80067,_80068,_80069,_80070,_80071,_80072,_80073,_80074,_80075,_80076,_80077,_80078,_80079,_80080,_80081,_80082,_80083,_80084,_80085,_80086,_80087,_80088,_80089,_80090,_80091,_80092,_80093,_80094,_80095,_80096,_80097,_80098,_80099,_80100,_80101,_80102,_80103,_80104,_80105,_80106,_80107,_80108,_80109,_80110,_80111,_80112,_80113,_80114,_80115,_80125,_80126,_80127,_80128,_80129,_80130,_80131,_80132,_80133,_80134,_80135,_80136,_80137,_80138,_80139,_80140,_80141,_80142,_80143,_80144,_80145,_80146,_80147,_80148,_80149,_80150,_80151
0,8922,8678,10,12,9289,10,7732,8549,2,2,9909,11474,12578,11,9,10,22464,34872,34961,2,11,10,56835,75738,95849,11,10,10,118753,73356,40545,85441,88655,91721,10,8,9,224,19608,109,17,16,618,5193,157,13,19,15,82,16,86,16,10,16,15100,30235,27876,2105,14547,5325,5993,2202,18,64,144,729,6323,10107,24,8,14,4124,12,7635,10,23,20,13,27,33,8016,18599,4364,2813,2000,15,5800,2213,2103,855,16,61,133,50,18,25,2758,9,9,8,9,11,10,2573106,2,2298098,10,2763548,11,806691,2,1281337,2,376985,2,734777,2,1072260,2,173778,2,389913,2,388832,2,1103921,2,1763450,2,2402628,2,2301100,2,2054187,2,1876049,2,2,2,131715,152314,12188,36037,45228,46210,9861,28253,146135,159685,146551,141477,160793,18602,48314,58383,50769,15434,37568,160965,167510,161343,26537,45974,101,459,923,9,57539,82826,405,1855,3373,2,68266,94710,761,3337,5702,2,2,2,6700,14284,108,208,5634,7500,2257,6255,33,36,8667,7851,109,107,103,94,92,89,87,109,99,98,87,84,82,75,79,66,66,60,66,60,59,110,113,68,39,7507,6707,109,122,6215,4253,1554,6304,33,33,4856,7605,107,104,89,88,89,81,73,108,96,77,72,70,68,57,39,31,29,30,30,34,24,106,133,90,39,22915,109,12026,2152,13950,50,42,45,42,44,42,43,18680,109,15773,178,12750,65,83,88,87,83,83,66,16,16,16,15,15,16,18,11,16,10,16,17,15,9,18,15,17,16,17,17,15,15,17,17,16,9,1596170,138688,130102,596703,4494,115311,41630,42601,427816,4065,1134809,169181,78162,148187,670707,328955,67602,131925


In [7]:
hashes = set()

for k, v in feature_hash.items():
    hashes |= v
print(f"Number of total hashes: {len(hashes)}")

Number of total hashes: 15673363


In [8]:
total_hash = df_hash.sum(axis=1).values[0]
print(f"Number of total hashes: {total_hash}")

Number of total hashes: 33829828


# Use pyspark to analyze

In [ ]:
columns = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        columns.append(line.split(' ')[1].strip())
print(columns)

In [ ]:
import metaspore as ms
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, explode, sum as spark_sum, countDistinct
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType

# Read column names from file
columns = []
with open('./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        columns.append(line.split(' ')[1].strip())
print(columns)

# Define Spark configuration and session
file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 1
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

spark_confs = {
    'spark.eventLog.enabled': 'true',
    'spark.executor.memory': '10g',
    'spark.driver.memory': '10g',
    "spark.driver.maxResultSize": "10g",
    "spark.sql.files.ignoreCorruptFiles": "true"
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=3,
                                     server_count=3,
                                     log_level='WARN',
                                     spark_confs=spark_confs)

# Load the dataset
train_dataset = ms.input.read_s3_csv(spark_session, 
                                     train_dataset_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=columns,
                                     multivalue_column_names=columns[:-1])

print(f"Number of orcs: {num_files}, total number of rows: {train_dataset.count()}")

# Define UDF to process rows
def process_row(column_values):
    feature_hashes = set()
    for value in column_values:
        if '\003' in value:
            hash_val, weight = value.split('\003')
        else:
            hash_val, weight = '', 0
        feature_hashes.add(hash_val)

    feature_values = len(column_values)
    return (feature_values, list(feature_hashes))

# Define schema for UDF output
schema = StructType([
    StructField("feature_values", IntegerType(), True),
    StructField("hashes", ArrayType(StringType()), True)
])

# Register the UDF
process_row_udf = udf(process_row, schema)

# Apply UDF on each column and process the dataset
for column in columns[:-1]:
    train_dataset = train_dataset.withColumn(f'{column}_processed', process_row_udf(col(column)))

train_dataset.show()


In [ ]:
# Extract the processed columns
result_columns = [f'{column}_processed' for column in columns[:-1]]
processed_df = train_dataset.select(*result_columns)
processed_df.show()

In [ ]:
# Select the desired row and column
row_value = train_dataset.select('_11002_processed').collect()[4][0]

# Print the value
print(row_value)

In [ ]:
# 1. Obtain the percentiles of feature_values using approxQuantile
from tqdm import tqdm
percentiles = [0.25, 0.5, 0.75, 1.0]

for col_name in tqdm(processed_df.columns, total=len(processed_df.columns)):
    feature_values_df = processed_df.select(col(f"{col_name}.feature_values").cast("double").alias("feature_values"))
    
    # Print schema to verify cast
    # feature_values_df.printSchema()
    
    # Show data to verify cast
    # feature_values_df.show()
    
    # Verify the data type of the column
    # data_type = feature_values_df.schema["feature_values"].dataType
    # print(f"Data type of feature_values in {col_name}: {data_type}")
    
    # # Check for null values or any anomalies in the column
    # null_count = feature_values_df.filter(col("feature_values").isNull()).count()
    # print(f"Number of null values in feature_values of {col_name}: {null_count}")
    
    # Obtain approximate percentiles
    try:
        percentile_values = feature_values_df.approxQuantile("feature_values", percentiles, 0.01)
        # print(f"Percentiles for {col_name}: {percentile_values}")
    except Exception as e:
        print(f"Error occurred while calculating percentiles for {col_name}: {e}")

# 2. Obtain the total number of different hash values for each column and across all columns
distinct_hash_counts = {}
all_hashes_df = []

print(f"Starting counting hashes")
for col_name in tqdm(processed_df.columns, total=len(processed_df.columns)):
    hashes_df = processed_df.select(explode(col(f"{col_name}.hashes")).alias("hashes"))
    distinct_count = hashes_df.select(countDistinct("hashes")).first()[0]
    distinct_hash_counts[col_name] = distinct_count
    all_hashes_df.append(hashes_df)

union_hashes_df = all_hashes_df[0]
for df in all_hashes_df[1:]:
    union_hashes_df = union_hashes_df.union(df)

total_distinct_hashes = union_hashes_df.select(countDistinct("hashes")).first()[0]

print(f"Distinct hash counts per column: {distinct_hash_counts}")
print(f"Total distinct hashes across all columns: {total_distinct_hashes}")

# Stop the Spark session
# spark.stop()

In [ ]:
# # 1. Obtain the percentiles of feature_values
# percentiles = [0.25, 0.5, 0.75, 1]
# for col_name in processed_df.columns:
#     feature_values_df = processed_df.select(col(f"{col_name}.feature_values").alias("feature_values"))
#     # feature_values_df.show()
#     feature_values_list = feature_values_df.select(collect_list("feature_values")).first()[0]
#     feature_values_list.sort()

#     def get_percentile(p):
#         k = (len(feature_values_list) - 1) * p
#         f = math.floor(k)
#         c = math.ceil(k)
#         if f == c:
#             return feature_values_list[int(k)]
#         return feature_values_list[int(f)] * (c - k) + feature_values_list[int(c)] * (k - f)
    
#     percentile_values = [get_percentile(p) for p in percentiles]
#     print(f"Percentiles for {col_name}: {percentile_values}")

# # 2. Obtain the total number of different hash values for each column and across all columns
# distinct_hash_counts = {}
# all_hashes = []

# for col_name in processed_df.columns:
#     hashes_df = processed_df.select(explode(col(f"{col_name}.hashes")).alias("hashes"))
#     # hashes_df.show()
#     distinct_count = hashes_df.select(countDistinct("hashes")).first()[0]
#     distinct_hash_counts[col_name] = distinct_count
#     all_hashes.extend(hashes_df.select("hashes").rdd.flatMap(lambda x: x).collect())

# total_distinct_hashes = len(set(all_hashes))

# print(f"Distinct hash counts per column: {distinct_hash_counts}")
# print(f"Total distinct hashes across all columns: {total_distinct_hashes}")
